## Project Phase 1 - Aviation Accident Data Integration
### Group 03:
- Tommaso Tragno - fc64699
- Manuel Cardoso - fc56274
- Chen Cheng - fc64872
- Cristian Tedesco - fc65149

#### Setup

In [5]:
import pandas as pd
import numpy as np
import json
import os
import requests

### 1. Load NTSB JSON Data

In [ ]:
PATH = 'data_sources/'
NTSB_DATA = 'ntsb-us-2003-2023.json'

with open(PATH+NTSB_DATA, "r", encoding="utf-8") as f:
    ntsb_raw_data = json.load(f)

# Each record is one accident/incident entry in a list
print(f"\n--- NTSB JSON loaded: {len(ntsb_raw_data)} total records found ---")

# Convert to a DataFrame (this will flatten top-level fields)
# For nested fields like "Vehicles", we might do a separate flatten later
df_ntsb = pd.json_normalize(ntsb_raw_data, 
                            meta=[
                                "Oid","MKey","Closed","CompletionStatus","HasSafetyRec",
                                "HighestInjury","IsStudy","Mode","NtsbNumber",
                                "OriginalPublishedDate","MostRecentReportType","ProbableCause",
                                "City","Country","EventDate","State","Agency","BoardLaunch",
                                "BoardMeetingDate","DocketDate","EventType","Launch","ReportDate",
                                "ReportNum","ReportType","AirportId","AirportName","AnalysisNarrative",
                                "FactualNarrative","PrelimNarrative","FatalInjuryCount","MinorInjuryCount",
                                "SeriousInjuryCount","InvestigationClass","AccidentSiteCondition",
                                "Latitude","Longitude","DocketOriginalPublishDate"
                            ],
                            record_path=["Vehicles"],  # This flattens out the 'Vehicles' array
                            record_prefix="Vehicles."
                           )

print("\n--- Flattened NTSB DataFrame (including Vehicles info): ---")

# Type Conversion
df_ntsb['EventDate'] = pd.to_datetime(df_ntsb['EventDate']).dt.tz_localize(None)
df_ntsb['User-ID'] = pd.to_numeric(df_ntsb['User-ID'], errors='coerce').fillna(0).astype(int)


df_ntsb = df_ntsb.map(lambda x: x.lower() if isinstance(x, str) else x) # make all appropriate values lowercase

# combines all injury counts to 1 column
df_ntsb['TotalInjuryCount'] = df_ntsb[['FatalInjuryCount', 'MinorInjuryCount', 'SeriousInjuryCount']].sum(axis=1)

# dropping unnecessary columns
df_ntsb.drop(columns=['AnalysisNarrative','FactualNarrative','PrelimNarrative','InvestigationClass','BoardLaunch'
                      ,'BoardMeetingDate','Launch','IsStudy','OriginalPublishedDate','DocketOriginalPublishDate'
                      ,'ReportType','ReportNum','ReportDate','MostRecentReportType','FatalInjuryCount','MinorInjuryCount'
                      ,'SeriousInjuryCount','DocketDate','Mode','HasSafetyRec','CompletionStatus','Closed'
                      ,'Vehicles.AircraftCategory','Vehicles.AmateurBuilt','Vehicles.EventID','Vehicles.AirMedical'
                      ,'Vehicles.AirMedicalType','Vehicles.flightScheduledType','Vehicles.flightServiceType'
                      ,'Vehicles.flightTerminalType','Vehicles.RegisteredOwner','Vehicles.RegulationFlightConductedUnder'
                      ,'Vehicles.RepGenFlag','Vehicles.RevenueSightseeing','Vehicles.SecondPilotPresent','Vehicles.Damage'
                      ], inplace=True) 

# dropping NaT entries from EventDate
df_ntsb = df_ntsb.dropna(subset=['EventDate'])


print(df_ntsb.head())

print("\n--- DataFrame Info ---")
df_ntsb.info()


--- NTSB JSON loaded: 22992 total records found ---

--- Flattened NTSB DataFrame (including Vehicles info): ---
   Vehicles.VehicleNumber Vehicles.DamageLevel Vehicles.ExplosionType  \
0                       1          substantial                   none   
1                       1          substantial                   none   
2                       1          substantial                   none   
3                       1          substantial                   none   
4                       1          substantial                   none   

  Vehicles.FireType Vehicles.SerialNumber       Vehicles.Make Vehicles.Model  \
0              none                  0702  cirrus design corp          sr22t   
1              none                 004ce   golden avio s r l       f30 brio   
2              none               4692139  piper aircraft inc     pa46r-350t   
3              none              79-30941            bellanca         17-30a   
4              none                 7452c      

## Data cleaning
1. Load the `.csv` and `.json` dataset;
2. Drop the rows that do not contains required data
3. Fill the `na` cells with a predefined value
4. Drop eventualy doplicates
5. Convert the string data into the proper data type

In [ ]:
PATH = 'data_sources'

# Load dataset into pandas dataframe
df_airline_traffic = pd.read_csv(f'{PATH}/u-s-airline-traffic-data.csv')
df_ntsb = pd.read_json(f'{PATH}/ntsb-us-2003-2023.json')

print('Check NA values presence before data validation')
print(f'Airline traffic data frame: {df_airline_traffic.isna().any().any()}')
print(f'NTSB data frame: {df_ntsb.isna().any().any()}')



print(df_ntsb.columns.tolist())
#print(df_ntsb.describe())  # Summary statistics
print(df_ntsb.info())  # Data types and missing values
#print(df_ntsb.isnull().sum())  # Check missing values

df_ntsb

Check NA values presence before data validation
Airline traffic data frame: False
NTSB data frame: True
['Oid', 'MKey', 'HighestInjury', 'NtsbNumber', 'ProbableCause', 'City', 'Country', 'EventDate', 'State', 'Agency', 'EventType', 'Vehicles', 'AirportId', 'AirportName', 'AccidentSiteCondition', 'Latitude', 'Longitude', 'TotalInjuryCount']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22992 entries, 0 to 22991
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Oid                    22992 non-null  object        
 1   MKey                   22992 non-null  int64         
 2   HighestInjury          22908 non-null  object        
 3   NtsbNumber             22992 non-null  object        
 4   ProbableCause          22800 non-null  object        
 5   City                   22992 non-null  object        
 6   Country                22992 non-null  object        
 7   EventDate    

,Oid,MKey,HighestInjury,NtsbNumber,ProbableCause,City,Country,EventDate,State,Agency,EventType,Vehicles,AirportId,AirportName,AccidentSiteCondition,Latitude,Longitude,TotalInjuryCount
0,67ee2dab017de3d12ee0378c,193595,none,cen24la079,None,davenport,usa,2023-12-31 17:40:00,ia,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",dvn,None,vmc,41.610278,-90.588361,0
1,67ee2dab017de3d12ee03793,193603,none,era24la084,the pilot’s improper recovery from a bounced l...,midland,usa,2023-12-31 16:13:00,va,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",hwy,warrenton/fauquier,vmc,38.586285,-77.710631,0
2,67ee2dab017de3d12ee0377f,193581,fatal,era24fa078,None,mooresville,usa,2023-12-31 14:13:00,nc,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",14a,lake norman airpark,vmc,35.624637,-80.912255,1
3,67ee2dab017de3d12ee03783,193585,serious,cen24la077,None,washington,usa,2023-12-30 17:00:00,mo,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",fyg,washington regional,vmc,38.587583,-90.993806,1
4,67ee2dab017de3d12ee03794,193605,none,cen24la081,the pilot’s failure to maintain directional co...,beaumont,usa,2023-12-29 16:27:00,tx,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",bpt,jack brooks rgnl,vmc,30.070603,-94.215837,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22987,67ee2d86017de3d12edf9ae9,56292,none,chi03la045,failure of the pilot-in-command to maintain di...,alton,usa,2003-01-03 13:00:00,il,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",aln,st louis regional,vmc,38.890277,-90.046112,0
22988,67ee2d86017de3d12edf9b2f,56373,none,mia03la044,the loss of engine power for undetermined reas...,jessup,usa,2003-01-03 10:12:00,ga,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Substant...",None,None,vmc,31.431667,-81.819999,0
22989,67ee2d86017de3d12edf9ad1,56264,fatal,ftw03fa071,the pilot's failure to maintain adequate airsp...,shreveport,usa,2003-01-02 14:46:00,la,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Destroye...",kdtn,shreveport downtown airport,vmc,32.593055,-93.765556,2
22990,67ee2d86017de3d12edf9acf,56260,serious,den03la027,the inadequate altitude chosen by the flight i...,milford,usa,2003-01-01 23:00:00,ut,ntsb,acc,"[{'VehicleNumber': 1, 'DamageLevel': 'Destroye...",None,None,vmc,38.283332,-112.900001,2


In [ ]:
# Code to filter to the date we want

# Debug: Check min and max dates
print("Earliest Date:", df_ntsb['EventDate'].min())
print("Latest Date:", df_ntsb['EventDate'].max())

# Define the date range (without timezone)
start_date = pd.to_datetime('2003-01-01')
end_date = pd.to_datetime('2023-12-31')

# Filter the dataset
filtered_df = df_ntsb[(df_ntsb['EventDate'] >= start_date) & (df_ntsb['EventDate'] <= end_date) & (df_ntsb['Country'] == 'usa')]
print(filtered_df['State'].tolist())
filtered_df
# Display results
#print(f"Total Records Found: {len(filtered_df)}")
#print(filtered_df[['EventDate', 'HighestInjury', 'Country']].sample(10))  # Show 50 random dates

### open-meteo API call test

In [ ]:
# Define the endpoint
endpoint = "https://archive-api.open-meteo.com/v1/archive"

# Define the parameters
params = {
    "latitude": 41.610278,
    "longitude": -90.588361,
    "start_date": "2023-12-31",
    "end_date": "2023-12-31",
    "hourly": ",".join([
        "temperature_2m",
        "relative_humidity_2m",
        "dew_point_2m",
        "pressure_msl",
        "surface_pressure",
        "precipitation",
        "rain",
        "snowfall",
        "cloud_cover",
        "cloud_cover_low",
        "cloud_cover_mid",
        "cloud_cover_high",
        "wind_speed_10m",
        "wind_speed_100m",
        "wind_direction_10m",
        "wind_direction_100m",
        "wind_gusts_10m",
        "weather_code",
        "snow_depth"
    ]),
    "timezone": "GMT"
}

# Make the request
response = requests.get(endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
    time_series = data["hourly"]["time"]
    try:
        idx = time_series.index("2023-12-31T17:00")
        selected_data = {k: v[idx] for k, v in data["hourly"].items() if k != "time"}
        print(f"Weather data at 2023-12-31T17:00Z:")
        for key, val in selected_data.items():
            print(f"{key}: {val}")
    except ValueError:
        print("Selected hour not found in response.")
else:
    print(f"Error: {response.status_code}")


{'latitude': 41.581722, 'longitude': -90.64935, 'generationtime_ms': 0.3064870834350586, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 228.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'dew_point_2m': '°C', 'pressure_msl': 'hPa', 'surface_pressure': 'hPa', 'precipitation': 'mm', 'rain': 'mm', 'snowfall': 'cm', 'cloud_cover': '%', 'cloud_cover_low': '%', 'cloud_cover_mid': '%', 'cloud_cover_high': '%', 'wind_speed_10m': 'km/h', 'wind_speed_100m': 'km/h', 'wind_direction_10m': '°', 'wind_direction_100m': '°', 'wind_gusts_10m': 'km/h', 'weather_code': 'wmo code', 'snow_depth': 'm'}, 'hourly': {'time': ['2023-12-31T00:00', '2023-12-31T01:00', '2023-12-31T02:00', '2023-12-31T03:00', '2023-12-31T04:00', '2023-12-31T05:00', '2023-12-31T06:00', '2023-12-31T07:00', '2023-12-31T08:00', '2023-12-31T09:00', '2023-12-31T10:00', '2023-12-31T11:00', '2023-12-31T12:00', '2023-12-31T13:00', '2023-12-31T14:00', '202